# 02 - Convex Model: Portfolio Optimization

This notebook builds a convex mean-variance optimizer, tests convexity (Hessian), runs the optimizer for several risk-aversion values, and saves results to the `4_Results` folder.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json

mu = pd.read_json("../4_Results/processed/mean_returns.json", typ="series")
cov = pd.read_csv("../4_Results/processed/cov_matrix.csv", index_col=0)
tickers = pd.read_csv("../4_Results/processed/selected_tickers.csv", header=None)[0].tolist()

mu.head(), cov.head()

In [ ]:
Sigma = cov.values
H = 2 * Sigma

eigenvalues = np.linalg.eigvals(H)
print("Eigenvalues:", eigenvalues)
print("Is PSD? →", np.all(eigenvalues >= -1e-8))

In [ ]:
import cvxpy as cp

n = len(tickers)
x = cp.Variable(n)

lam = 10

objective = cp.Minimize(cp.quad_form(x, Sigma) - lam * mu.values @ x)

constraints = [cp.sum(x) == 1, x >= 0, x <= 0.3]

problem = cp.Problem(objective, constraints)
problem.solve()

weights = x.value
weights

In [ ]:
plt.figure(figsize=(12,5))
sns.barplot(x=tickers, y=weights)
plt.xticks(rotation=45)
plt.title("Optimal Portfolio Weights (Convex Model)")
plt.show()

In [ ]:
expected_return = mu.values @ weights
risk = weights.T @ Sigma @ weights

print("Expected Return:", expected_return)
print("Risk:", risk)

In [ ]:
plt.figure(figsize=(7,5))
plt.scatter(risk, expected_return, s=100)
plt.xlabel("Portfolio Risk")
plt.ylabel("Expected Return")
plt.title("Convex Model: Risk vs Return")
plt.grid(True)
plt.show()